# Projeto de Processamento de Linguagem Natural

Neste projeto de NLP, você tentará classificar as avaliações do Yelp em categorias de 1 ou 5 estrelas com base no conteúdo do texto nas avaliações. Utilizaremos os métodos de pipeline para tarefas mais complexas.

Utilizaremos o [Yelp Review Data Set](https://www.kaggle.com/c/yelp-recsys-2013) do Kaggle.

Cada observação neste conjunto de dados é uma revisão de um determinado negócio por um determinado usuário.

A coluna "stars" é o número de estrelas (1 a 5) atribuídas pelo revisor à empresa.

A coluna "cool" é o número de votos "cool" que esta avaliação recebeu de outros usuários do Yelp.

Todas as resenhas começam com 0 votos "cool" e não há limite para quantos votos "cool" uma resenha pode receber. Em outras palavras, é uma classificação da avaliação em si, não uma classificação do negócio.

As colunas "useful" e "funny" são semelhantes à coluna "cool".

## Importações
**Importe os usual suspects :)**

In [ ]:
import numpy as np
import pandas as pd

## Os dados

**Leia o arquivo yelp.csv e defina-o como um dataframe chamado yelp.**

In [ ]:
# Esse trecho do código é obrigatório para quem estiver fazendo tudo pelo colab
# Caso você esteja utilizando o jupyter pode comentar/apagar
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir("drive/My Drive/Colab Notebooks/IA/20_Natural_Language_Processing")
os.listdir()

In [ ]:
yelp = pd.read_csv('yelp.csv')

**Verifique os métodos head, info e describe no yelp.**

In [ ]:
yelp.head()

In [ ]:
yelp.info()

In [ ]:
yelp.describe()

**Crie uma nova coluna chamada "text length", que é o número de palavras na coluna de texto.**

In [ ]:
yelp['text length'] = yelp['text'].apply(len)

# EDA

**Importe as bibliotecas de visualização de dados, caso ainda não o tenha feito.**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

**Use o FacetGrid da biblioteca seaborn para criar uma grade de 5 histogramas do comprimento do texto com base na quantidade de estrelas recebidas. Consulte a documentação do seaborn para obter dicas sobre isso**

In [ ]:
g = sns.FacetGrid(yelp,col='stars')
g.map(plt.hist,'text length')

**Crie um boxplot de tamanho de texto para cada categoria de estrela (quantidade de estrelas recebidas).**

In [ ]:
sns.boxplot(x='stars',y='text length',data=yelp,palette='rainbow')

**Crie um countplot do número de ocorrências para cada tipo de classificação por estrelas.**

In [ ]:
sns.countplot(x='stars',data=yelp,palette='rainbow')

**Use groupby para obter os valores médios das colunas numéricas, você deve ser capaz de criar este dataframe com a operação:**

`stars = yelp.groupby('stars').mean()`

In [ ]:
stars = yelp.groupby('stars').mean()
stars

**Use o método corr() nesse dataframe groupby:**

In [ ]:
stars.corr()

**Em seguida, use seaborn para criar um mapa de calor baseado nesse dataframe .corr():**

In [ ]:
sns.heatmap(stars.corr(),cmap='coolwarm',annot=True)

## Tarefa de classificação de NLP

Vamos passar para a tarefa real. Para tornar as coisas um pouco mais fáceis, iremos utilizar apenas avaliações de 1 ou 5 estrelas.

**Crie um dataframe chamado yelp_class que contém as colunas do yelp dataframe, mas apenas para as avaliações de 1 ou 5 estrelas.**

In [ ]:
yelp_class = yelp[(yelp.stars==1) | (yelp.stars==5)]

**Crie dois objetos X e y. X será a coluna 'text' de yelp_class e y será a coluna 'stars' de yelp_class, suas características (features) e alvo/rótulos (target/labels), respectivamente**

In [ ]:
X = yelp_class['text']
y = yelp_class['stars']

**Importe CountVectorizer e crie um objeto CountVectorizer.**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

**Use o método fit_transform no objeto CountVectorizer e passe em X (coluna 'texto'). Salve este resultado substituindo X.**

In [ ]:
X = cv.fit_transform(X)

## Divisão de teste/treinamento

Vamos dividir nossos dados em dados de treinamento e teste.

**Use train_test_split para dividir os dados em X_train, X_test, y_train, y_test. Use test_size=0.3 e random_state=101**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=101)

## Treinando um Modelo

**Importe MultinomialNB, crie uma instância do estimador e chame de nb **

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

**Agora ajuste nb usando os dados de treinamento.**

In [ ]:
nb.fit(X_train,y_train)

## Previsões e avaliações

**Use o método predict de nb para prever rótulos de X_test.**

In [ ]:
predictions = nb.predict(X_test)

**Crie uma matriz de confusão (confusion matrix) e um relatório de classificação (classification report) usando essas previsões e y_test**

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

**Excelente! Vamos ver o que acontece se tentarmos incluir TF-IDF nesse processo usando um pipeline.**

# Usando processamento de texto

**Importar TfidfTransformer de sklearn.**

In [ ]:
from sklearn.feature_extraction.text import  TfidfTransformer

** Importar Pipeline de sklearn.**

In [ ]:
from sklearn.pipeline import Pipeline

**Agora crie um pipeline com os seguintes passos:CountVectorizer(), TfidfTransformer(),MultinomialNB()**

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # strings para contagens inteiras de token
    ('tfidf', TfidfTransformer()),  # contagens inteiras para pontuações TF-IDF ponderadas
    ('classifier', MultinomialNB()),  # treine em vetores TF-IDF com classificador Naive Bayes
])

## Usando o pipeline

**Hora de usar o pipeline! Lembre-se de que este pipeline já contém todas as etapas de pré-processamento, o que significa que precisaremos dividir novamente os dados originais (lembre-se de que substituímos X como a versão CountVectorized. O que precisamos é apenas o texto **

### Divisão treino/teste

**Refaça a divisão teste/treino no objeto yelp_class.**

In [ ]:
X = yelp_class['text']
y = yelp_class['stars']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=101)

**Agora ajuste o pipeline aos dados de treinamento. Lembre-se de que você não pode usar os mesmos dados de treinamento da última vez porque esses dados já foram vetorizados. Precisamos passar apenas o texto e os rótulos**

In [ ]:
# May take some time
pipeline.fit(X_train,y_train)

### Previsões e avaliação

**Agora use o pipeline para prever a partir do X_test e criar um relatório de classificação e uma matriz de confusão. Você deve observar resultados estranhos.**

In [ ]:
predictions = pipeline.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

Aparentemente o Tf-Idf piorou as coisas!